In [0]:
import numpy as np
import networkx as nx
import numpy.linalg as la
import scipy.cluster.vq as vq
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import statistics
import csv
import warnings
import random
warnings.filterwarnings('ignore')

In [0]:
def block_planted_clustering_quality(labels, num_members_in_cluster):
  score = 0
  label_first = 0
  label_second = 1
  k = 0
  for i in range(0,num_members_in_cluster):
    if labels[i] == 1:
      k += 1
  if (k >= num_members_in_cluster/2):
    label_first = 1
    label_second = 0
  for i in range(0, num_members_in_cluster):
    if labels[i] == label_first:
      score += 1
  for i in range(num_members_in_cluster, 2 * num_members_in_cluster):
    if labels[i] == label_second:
      score += 1
  score = score / (num_members_in_cluster * 2)
  return score

In [0]:
def stoer_wagner_cut_clustering(graph, num_clusters, num_members_in_cluster):
    px, pk = nx.stoer_wagner(graph, weight = 'weight')
    #print(pk)
    score_ = 0
    labels_ = [None] * (num_clusters * num_members_in_cluster)
    for i in pk[0]:
        labels_[i] = 0
    for i in pk[1]:
        labels_[i] = 1
    score_ = block_planted_clustering_quality(labels_, num_members_in_cluster)
    return score_, labels_

In [0]:
def create_weighted_graph(graph, p_in, p_out, num_groups = 2, distribution = [80, 110] ):
    new_edges =[]
    weighted_graph = nx.Graph()

    for edge in edges:
        src = edge[0]
        dst = edge[1]
        if (src < graph.number_of_nodes()/2 and dst < graph.number_of_nodes()/2) or (src > graph.number_of_nodes()/2 and dst > graph.number_of_nodes()/2):
            #new_edges.append((edge[0], edge[1], p_in * random.uniform(distribution[0], distribution[-1])))
            weighted_graph.add_edge(src,dst, weight=p_in * random.uniform(distribution[0], distribution[-1]))
        else:
            #new_edges.append((edge[0], edge[1], p_out * random.uniform(distribution[0], distribution[-1])))
            weighted_graph.add_edge(src,dst, weight=p_out * random.uniform(distribution[0], distribution[-1]))
    return weighted_graph

In [0]:
def min_cut_clustering_to_get_p_out(start_p_out = 0.0005, p_in = 1, num_groups = 2, num_members = 100,
                                    step = 0.0005, num_iterates = 10000, num_repeats = 20, file_name = "fixed_p_out.csv"):
  p_out = start_p_out
  result = []
  cur_i = 0
  while (p_out <= 0.005 and cur_i < num_iterates):
    scores = []
    for i in range(0, num_repeats):
      graph_ = nx.planted_partition_graph(num_groups,num_members,p_in,p_out)
      edges = graph_.edges()
      edges_number = len(edges)
      distribution = [80, 110]
      new_edges =[]
      weighted_graph_ = create_weighted_graph(graph_, p_in, p_out)
      score, labels = stoer_wagner_cut_clustering(weighted_graph_, num_groups, num_members)
      scores.append(score)
    result.append([p_out, statistics.mean(scores)])
    p_out += step
    cur_i += 1
    print('Iter: ' + str(cur_i))
  with open(file_name, "w", newline ='') as f:
      writer = csv.writer(f)
      writer.writerow(['p_in', str(p_in)])
      writer.writerow(['p_out', 'score'])
      for item in result:
        writer.writerow([str(item[0]), str(item[1])])
  return result

In [0]:
def p_out_grid_wmin_cut(file_name = "weighted_min_cut_p_out.csv", power = 1,  p_out_range = [0.09, 0.11, 0.001], 
                        p_in = 1, num_groups = 2, num_members = 100, num_repeats = 10):
  p_out_step = p_out_range[-1]
  p_out = p_out_range[0]
  result = []
  cur_i = 0
  max_cur =int( (p_out_range[1] - p_out_range[0]) / p_out_step )
  while p_out < p_out_range[1]:
    scores = []
    cur_i += 1 
    for i in range(0, num_repeats):
      graph_ = nx.planted_partition_graph(num_groups,num_members,p_in,p_out)
      edges = graph_.edges()
      edges_number = len(edges)
      koef = pow(float(p_out / p_in), power)
      distribution_in = [80, 110]
      distribution_out = [distribution_in[0] * koef, distribution_in[-1] * koef]
      new_edges =[]
      weighted_graph = nx.Graph()
      for edge in edges:
          src = edge[0]
          dst = edge[1]
          if (src < num_members and dst < num_members) or (src >= num_members and dst >= num_members):
              weighted_graph.add_edge(src,dst, weight= random.uniform(distribution_in[0], distribution_in[-1]))
          else:
              weighted_graph.add_edge(src,dst, weight=  random.uniform(distribution_out[0], distribution_out[-1]))
      score, labels = stoer_wagner_cut_clustering(weighted_graph, num_groups, num_members)
      scores.append(score)
    result.append([p_out, statistics.mean(scores)])
    p_out += p_out_step
    if cur_i % 5 == 0:
      print((cur_i / max_cur)*100,'%') 
  with open(file_name, "w", newline ='') as f:
      writer = csv.writer(f)
      writer.writerow(['p_in', str(p_in)])
      writer.writerow(['p_out', 'score'])
      for item in result:
        writer.writerow([str(item[0]), str(item[1])])
  return result
  

In [0]:
p_out_grid_wmin_cut(power = 2)

25.0 %
50.0 %
75.0 %
100.0 %


[[0.09, 1.0],
 [0.091, 1.0],
 [0.092, 1.0],
 [0.093, 1.0],
 [0.094, 1.0],
 [0.095, 1.0],
 [0.096, 1.0],
 [0.097, 1.0],
 [0.098, 1.0],
 [0.099, 1.0],
 [0.1, 1.0],
 [0.101, 1.0],
 [0.10200000000000001, 1.0],
 [0.10300000000000001, 1.0],
 [0.10400000000000001, 1.0],
 [0.10500000000000001, 1.0],
 [0.10600000000000001, 1.0],
 [0.10700000000000001, 1.0],
 [0.10800000000000001, 1.0],
 [0.10900000000000001, 1.0]]

In [0]:
num_groups = 2
num_members = 100
p_in = 1
p_out =  0.1
graph_ = nx.planted_partition_graph(num_groups,num_members,p_in,p_out)
edges = graph_.edges()
edges_number = len(edges)
power = 1
koef = pow(float(p_out / p_in), power)
distribution_in = [80, 110]
distribution_out = [distribution_in[0] * koef, distribution_in[-1] * koef]
new_edges =[]
weighted_graph = nx.Graph()
for edge in edges:
    src = edge[0]
    dst = edge[1]
    if (src < num_members and dst < num_members) or (src >= num_members and dst >= num_members):
        #new_edges.append((edge[0], edge[1], p_in * random.uniform(distribution[0], distribution[-1])))
        weighted_graph.add_edge(src,dst, weight= random.uniform(distribution_in[0], distribution_in[-1]))
    else:
        #new_edges.append((edge[0], edge[1], p_out * random.uniform(distribution[0], distribution[-1])))
        weighted_graph.add_edge(src,dst, weight=  random.uniform(distribution_out[0], distribution_out[-1]))
#weighted_graph.add_weighted_edges_from(new_edges)
#A = nx.adjacency_matrix(weighted_graph)
#print(A.todense())
cut_size,  s_sda = nx.stoer_wagner(weighted_graph, weight = 'weight')
print('cut_size ', cut_size)
score = 1
success_iters = -1
success_iters += 1
score, labels = stoer_wagner_cut_clustering(weighted_graph, num_groups, num_members)
print('score ', score, '      ', 'labels ', labels)

cut_size  8971.248183452828
score  1.0        labels  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [0]:
min_cut_clustering_to_get_p_out()

KeyboardInterrupt: 